# Parallel matrix multiplication

Note: distributed initialization, doesn't use files.

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
# Start PyCOMPSs runtime with graph and tracing enabled
ipycompss.start(graph=True, trace=True, monitor=2000)

In [ ]:
from pycompss.api.task import task
from pycompss.api.parameter import *
import numpy as np
import os

## Task definitions

In [ ]:
@task(returns=list)
def createBlock(BSIZE, res, MKLProc):
    os.environ["MKL_NUM_THREADS"]=str(MKLProc)
    if res:
        block = np.array(np.zeros((BSIZE, BSIZE)), dtype=np.double, copy=False)
    else:
        block = np.array(np.random.random((BSIZE, BSIZE)), dtype=np.double,copy=False)
    mb = np.matrix(block, dtype=np.double, copy=False)
    return mb

In [ ]:
@task(c=INOUT)
def multiply(a, b, c, MKLProc):
    os.environ["MKL_NUM_THREADS"]=str(MKLProc)
    c += a * b

## Functions

In [ ]:
def initialize_variables(MKLProc):
    for matrix in [A, B]:
        for i in range(MSIZE):
            matrix.append([])
            for j in range(MSIZE):
                mb = createBlock(BSIZE, False, MKLProc)
                matrix[i].append(mb)
    for i in range(MSIZE):
        C.append([])
        for j in range(MSIZE):
            mb = createBlock(BSIZE, True, MKLProc)
            C[i].append(mb)

## MAIN Code

Parameters (that can be configured in the following cell):
* MSIZE: Matrix size (default: 16)
* BSIZE: Block size (default: 32)
* MKLProc: Number of MKL processes (default: 1)

In [ ]:
import time
begginingTime = time.time()
import sys
from pycompss.api.api import compss_wait_on, compss_barrier

MSIZE = 16
BSIZE = 32
MKLProc = 1
A = []
B = []
C = []

startTime = time.time()

initialize_variables(MKLProc)

compss_barrier()

initTime = time.time() - startTime
startMulTime = time.time()

for i in range(MSIZE):
    for j in range(MSIZE):
        for k in range(MSIZE):
            multiply(A[i][k], B[k][j], C[i][j], MKLProc)

C = compss_wait_on(C)

mulTime = time.time() - startMulTime
mulTransTime = time.time() - startMulTime
totalTime = time.time() - startTime
totalTimeWithImports = time.time() - begginingTime

In [ ]:
# Plot Result
print("PARAMS:------------------")
print("MSIZE:{}".format(MSIZE))
print("BSIZE:{}".format(BSIZE))
print("initT:{}".format(initTime))
print("multT:{}".format(mulTime))
print("mulTransT:{}".format(mulTransTime))
print("totalTime:{}".format(totalTime))
import pprint
pprint.pprint(C)

In [ ]:
ipycompss.stop()